# ABS Detailed Labour Force 6291

## Python set-up

In [1]:
# analytic imports
import matplotlib.pyplot as plt
import pandas as pd
from abs_data_capture import (
    AbsLandingPage,
    get_abs_data,
    get_fs_constants,
    get_identifier,
    metacol,
    get_plot_constants,
)

# local imports
from plotting import (
    clear_chart_dir,
    line_plot,
    recalibrate_series,
    set_chart_dir,
)

# pandas display settings
pd.options.display.max_rows = 99999
pd.options.display.max_columns = 999

# show charts within this notebook
SHOW = False

## Get data from ABS

In [2]:
landing_page = AbsLandingPage(
    theme="labour",
    parent_topic="employment-and-unemployment",
    topic="labour-force-australia-detailed",
)

In [3]:
# --- monthly
abs_m = get_abs_data(landing_page)
source, CHART_DIR, Cat_ID, meta_m = get_fs_constants(abs_m, landing_page)
RECENT, plot_times, plot_tags = get_plot_constants(meta_m)
print(f"Monthly tables: {meta_m['Table'].unique()}")

# --- quarterly
abs_q = get_abs_data(landing_page, 2)
source, CHART_DIR, Cat_ID, meta_q = get_fs_constants(abs_q, landing_page)
RECENT, plot_times, plot_tags = get_plot_constants(meta_q)
print(f"Quarterly tables: {meta_q['Table'].unique()}")

Monthly tables: ['1' '2' '3' '8' '9' '10' '14a' '14b' '14c' '14d' '14e' '16' '16b' '16c'
 '20a' '20b' '20c']
Quarterly tables: ['4' '5' '6' '7' '11' '12' '13' '17' '19' '23a' '23b' '24a' '24b' '25a'
 '25b' '26a' '26b' '27' '28' '29a' '29b']


## Plot

In [4]:
def chart_settings(chart_dir: str = CHART_DIR) -> None:
    """Set up charting parameters."""

    set_chart_dir(chart_dir)
    clear_chart_dir(chart_dir)
    plt.style.use("fivethirtyeight")


chart_settings()

### Monthly by Age

In [5]:
def monthly_by_age() -> None:
    """Produce monthly by age charts."""

    table = "1"
    data = abs_m[table]
    series_type = "Original"

    ages = [
        "15-24 years",
        "25-34 years",
        "35-44 years",
        "45-54 years",
        "55-64 years",
    ]

    token = "By Age"
    plots = [
        f"> {token} ;  Participation rate ;  Persons ;",
        f"> {token} ;  Unemployment rate ;  Persons ;",
    ]

    for plot in plots:
        combined = {}
        for age in ages:
            desc = plot.replace(token, age)
            ident, units = get_identifier(meta_m, desc, series_type, table)
            combined[age] = data[ident]

        title = f"Australia: {plot}"
        line_plot(
            pd.DataFrame(combined),
            title=title,
            ylabel=units,
            rfooter=f"{source} {table}",
            lfooter=f"{series_type.capitalize()} series",
            show=SHOW,
        )


monthly_by_age()

### Quarterly by sector

In [6]:
def quarterly_by_sector() -> None:
    """Produce quarterly by sector charts."""

    table = "4"
    data_table = abs_q[table]
    series_type = "Seasonally Adjusted"
    selection = meta_q[
        (meta_q["Series Type"] == series_type) & (meta_q["Table"] == table)
    ]

    for _, series in selection.iterrows():
        title = series["Data Item Description"].replace(" ;", "").replace("  ", " ")
        series_id, units = series["Series ID"], series["Unit"]
        series, units = recalibrate_series(data_table[series_id], units)
        line_plot(
            series,
            title=title,
            ylabel=units,
            rfooter=f"{source} {table}",
            lfooter=f"{series_type.capitalize()} series",
            show=SHOW,
        )


quarterly_by_sector()

### Monthly by duration of unemployment

In [7]:
def monthly_by_ue_duration() -> None:
    """Produce monthly UE duration charts."""

    table = "14a"
    df = abs_m[table]
    series_type = "Original"

    cohorts = [
        "2 years and over",
        "1-2 years",
        "6-12 months",
        "3-6 months",
        "1-3 months",
        "under 1 month",
    ]

    categories = ["Unemployed total ;  Persons ;"]
    period = 12
    for category in categories:
        raw = pd.DataFrame()
        trend = pd.DataFrame()
        for cohort in cohorts:
            rows = meta_m[
                meta_m[metacol.did].str.contains(category)
                & meta_m[metacol.did].str.contains(cohort)
            ]
            assert len(rows) == 1
            select = rows["Series ID"].values[0]
            units = rows["Unit"].values[0]
            raw[cohort] = df[select]
            trend[cohort] = df[select].rolling(period).mean()

        title = (
            category.replace(" ; ", " ").replace(" ;", " ").replace("  ", " ").title()
        )
        lfooter = f"Australia. {series_type.capitalize()} series. "

        for data, t in zip([raw, trend], [title, f"{title}, {period} month ave."]):
            line_plot(
                data,
                title=t,
                ylabel=units,
                rfooter=f"{source} {table}",
                lfooter=lfooter,
                show=SHOW,
            )


monthly_by_ue_duration()

##  Household structure

In [11]:
abs_fm = get_abs_data(landing_page, zip_table=1, verbose=True)
source, _chart_dir, _cat_id, meta_fm = get_fs_constants(abs_fm, landing_page)


In get_abs_data() zip_table=1 verbose=True
About to get data on: Labour Force Australia Detailed 
Found: 89 items of type .xlsx: ['6291001.xlsx', '6291002.xlsx', '6291003.xlsx', 'LM1.xlsx', 'LM3.xlsx', 'LM3a.xlsx', 'LM4.xlsx', 'LM5.xlsx', 'LM7.xlsx', 'LM9.xlsx', '6291024a.xlsx', '6291024b.xlsx', '6291025a.xlsx', '6291025b.xlsx', 'LQ1.xlsx', 'LQ2.xlsx', '6291004.xlsx', '6291005.xlsx', '6291006.xlsx', '6291007.xlsx', '6291011.xlsx', '6291012.xlsx', 'EQ03.xlsx', 'EQ06.xlsx', 'EQ07a.xlsx', 'EQ07b.xlsx', 'EQ08.xlsx', 'EQ09.xlsx', 'EQ10.xlsx', 'EQ11.xlsx', 'EQ12.xlsx', 'EQ13.xlsx', 'EQ14.xlsx', '6291026a.xlsx', '6291026b.xlsx', '6291027.xlsx', '6291028.xlsx', '6291009.xlsx', '6291010.xlsx', 'EM1a.xlsx', 'EM1b.xlsx', 'EM2a.xlsx', 'EM2b.xlsx', 'EM3a.xlsx', 'EM3b.xlsx', 'EM4a.xlsx', 'EM4b.xlsx', 'EM5a.xlsx', 'EM5b.xlsx', '6291014a.xlsx', '6291014b.xlsx', '6291014c.xlsx', '6291014d.xlsx', '6291014e.xlsx', '6291020a.xlsx', '6291020b.xlsx', '6291020c.xlsx', 'UM2.xlsx', 'UM3.xlsx', 'UQ2a.xlsx', 'UQ

ChunkedEncodingError: ('Connection broken: IncompleteRead(76930483 bytes read, 195320351 more expected)', IncompleteRead(76930483 bytes read, 195320351 more expected))

## Finished

In [8]:
# watermark
%load_ext watermark
%watermark -u -n -t -v -iv -w

Last updated: Wed Apr 10 2024 16:59:17

Python implementation: CPython
Python version       : 3.12.2
IPython version      : 8.22.2

pandas    : 2.2.1
matplotlib: 3.8.4

Watermark: 2.4.3



In [9]:
print("Finished")

Finished
